In [1]:
import sys
from sklearn import preprocessing, model_selection
import tensorflow as tf
import pandas as pd
import numpy as np
import joblib
import argparse
from argparse import RawTextHelpFormatter, RawDescriptionHelpFormatter
import os
from scipy import stats
from tqdm import tqdm
import csv
from tensorflow.python.client import device_lib 
from numba import cuda
import gc

In [37]:
# from google.colab import drive
# drive.mount('/content/drive')

In [38]:
def rmse(y_true, y_pred):
    dev = np.square(y_true.ravel() - y_pred.ravel())
    return np.sqrt(np.sum(dev) / y_true.shape[0])


def pcc(y_true, y_pred):
    p = stats.pearsonr(y_true, y_pred)
    return p[0]


def pcc_rmse(y_true, y_pred):
    global alpha

    dev = np.square(y_true.ravel() - y_pred.ravel())
    r = np.sqrt(np.sum(dev) / y_true.shape[0])

    p = stats.pearsonr(y_true, y_pred)[0]

    return (1-p)*alpha + r * (1 - alpha)


def PCC_RMSE(y_true, y_pred):
    global alpha

    fsp = y_pred - tf.keras.backend.mean(y_pred)
    fst = y_true - tf.keras.backend.mean(y_true)

    devP = tf.keras.backend.std(y_pred)
    devT = tf.keras.backend.std(y_true)

    r = tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true), axis=-1))

    p = 1.0 - tf.keras.backend.mean(fsp * fst) / (devP * devT)

    #p = tf.where(tf.is_nan(p), 0.25, p)

    return alpha * p + (1 - alpha) * r


def RMSE(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true), axis=-1))


def PCC(y_true, y_pred):
    fsp = y_pred - tf.keras.backend.mean(y_pred)
    fst = y_true - tf.keras.backend.mean(y_true)

    devP = tf.keras.backend.std(y_pred)
    devT = tf.keras.backend.std(y_true)

    return tf.keras.backend.mean(fsp * fst) / (devP * devT)

In [39]:
def create_model(input_size, lr=0.0001, maxpool=True, dropout=0.1):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(32, kernel_size=4, strides=1,
                                     padding="valid", input_shape=input_size))
    model.add(tf.keras.layers.Activation("relu"))
    if maxpool:
        model.add(tf.keras.layers.MaxPooling2D(
            pool_size=2,
            strides=2,
            padding='same',  # Padding method
        ))

    model.add(tf.keras.layers.Conv2D(64, 4, 1, padding="valid"))
    model.add(tf.keras.layers.Activation("relu"))
    if maxpool:
        model.add(tf.keras.layers.MaxPooling2D(
            pool_size=2,
            strides=2,
            padding='same',  # Padding method
        ))

    model.add(tf.keras.layers.Conv2D(128, 4, 1, padding="valid"))
    model.add(tf.keras.layers.Activation("relu"))
    if maxpool:
        model.add(tf.keras.layers.MaxPooling2D(
            pool_size=2,
            strides=2,
            padding='same',  # Padding method
        ))

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(400, kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(200,
                                    kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(100, kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(dropout))

    #model.add(tf.keras.layers.Dense(20, kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    #model.add(tf.keras.layers.Activation("relu"))
    #model.add(tf.keras.layers.BatchNormalization())
    #model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01), ))
    model.add(tf.keras.layers.Activation("relu"))

    sgd = tf.keras.optimizers.SGD(lr=lr, momentum=0.9, decay=1e-6, )
    model.compile(optimizer=sgd, loss=PCC_RMSE, metrics=['mse'])

    return model

In [40]:
reshape = [60,169,1]

In [41]:
train_file = "train_test_validate_set/train.csv"
val_file = "train_test_validate_set/validate.csv"
test_file = "train_test_validate_set/test.csv"
path = ""

In [42]:
Xtrain, ytrain = None, []
df = pd.read_csv(path + train_file,index_col=0,header = 0,names = None).dropna()
for index,row in tqdm(df.iterrows()):
  if(index==0):
    Xtrain = row.values[1:-2].reshape(1,-1)
  else:
    Xtrain = np.concatenate((Xtrain,row.values[1:-2].reshape(1,-1)), axis=0)
  ytrain = ytrain + [row.values[-1]]


3243it [39:19,  1.37it/s]


In [43]:
Xval, yval = None, []
df = pd.read_csv(path + val_file,index_col=0,header = 0,names = None).dropna()
# print(df.head())
index = 0
for _,row in tqdm(df.iterrows()):
  if(index==0):
    Xval = row.values[1:-2].reshape(1,-1)
  else:
    Xval = np.concatenate((Xval,row.values[1:-2].reshape(1,-1)), axis=0)
  yval = yval + [row.values[-2]]
  index = index + 1

Xtest, ytest = None, []
df = pd.read_csv(path + test_file,index_col=0,header = 0,names = None).dropna()
index = 0
for _,row in tqdm(df.iterrows()):
  if(index==0):
    Xtest = row.values[1:-2].reshape(1,-1)
  else:
    Xtest = np.concatenate((Xtest,row.values[1:-2].reshape(1,-1)), axis=0)
  ytest = ytest + [row.values[-2]]
  index = index + 1

1000it [03:36,  4.61it/s]
350it [00:21, 16.41it/s]


In [44]:
scaler = preprocessing.StandardScaler()
X_train_val = np.concatenate((Xtrain, Xval), axis=0)
scaler.fit(X_train_val)

StandardScaler()

In [45]:
Xtrain = scaler.transform(Xtrain).reshape(-1, reshape[0],reshape[1],reshape[2])
Xval = scaler.transform(Xval).reshape(-1, reshape[0],reshape[1],reshape[2])
Xtest = scaler.transform(Xtest).reshape(-1, reshape[0],reshape[1],reshape[2])
ytrain = np.array(ytrain).reshape(-1, 1)
yval = np.array(yval).reshape(-1, 1)
ytest = np.array(ytest).reshape(-1, 1)

In [46]:
print(device_lib.list_local_devices()) 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4618497472360326996
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10555126996340919395
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10875295488
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 3220129135513849261
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:86:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 8107127574926832468
physical_device_desc: "device: XLA_GPU device"
]


In [ ]:
log = []
stop = [[0,999.9], ]
batch_size = 128
epochs = 150
global alpha
alpha = 0
patience = 10
path_model = "models/"
path_log = "logs/"
model_name = "model4.h5"
log_name = "log4.csv"
delta_loss = 0.0001
lr = 0.0005
dropout = 0
maxpool = False
model = create_model((reshape[0], reshape[1], reshape[2]),
                                 lr=lr, dropout=dropout, maxpool=maxpool)

In [ ]:
# cuda.select_device(0)
# cuda.close()

In [ ]:
with open('generation.csv', mode='a') as generation_file:
    generation_writer = csv.writer(generation_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    generation_writer.writerow([model_name, log_name, lr, dropout, maxpool, alpha, batch_size])

for e in range(1, epochs+1):
    model.fit(Xtrain, ytrain, validation_data=(Xval, yval),batch_size=batch_size, epochs=1, verbose=1)

    ytrain_pred = model.predict(Xtrain).ravel()
    loss = pcc_rmse(ytrain.ravel(), ytrain_pred)
    pcc_train = pcc(ytrain.ravel(), ytrain_pred)
    rmse_train = rmse(ytrain.ravel(), ytrain_pred)

    yval_pred = model.predict(Xval).ravel()
    loss_val = pcc_rmse(yval.ravel(), yval_pred)
    pcc_val = pcc(yval.ravel(), yval_pred)
    rmse_val = rmse(yval.ravel(), yval_pred)

    ytest_pred = model.predict(Xtest).ravel()
    loss_test = pcc_rmse(ytest.ravel(), ytest_pred)
    pcc_test = pcc(ytest.ravel(), ytest_pred)
    rmse_test = rmse(ytest.ravel(), ytest_pred)

    log.append([e, loss, pcc_train, rmse_train,
                    loss_val, pcc_val, rmse_val,
                    loss_test, pcc_test, rmse_test])
    logs    = pd.DataFrame(log, columns=['epoch', 'loss', 'pcc_train', 'rmse_train',
                                             'loss_val', 'pcc_val', 'rmse_val',
                                             'loss_test', 'pcc_test', 'rmse_test'])

    print("EPOCH:%d Loss:%.3f RMSE:%.3f PCC:%.3f LOSS_VAL:%.3f RMSE:%.3f PCC:%.3f LOSS_TEST:%.3f RMSE_TEST:%.3f PCC_TEST:%.3f"%
          (e, loss, rmse_train, pcc_train, loss_val, rmse_val, pcc_val, loss_test, rmse_test, pcc_test ))            

    if(stop[-1][1] - loss_val >= delta_loss):
        print("Model improve from %.3f to %.3f. Save model to %s."% (stop[-1][1], loss_val, path_model + model_name))
        model.save(path_model + model_name)
        stop.append([e, loss_val])
    else:
        if(e - stop[-1][0] >= patience):
            print("Get best model at epoch = %d." % stop[-1][0])
            break
            
logs.to_csv(path_log + log_name)